In [7]:
import mib_v2_3_3 as mb 
import numpy as np
import glob
import tpproc as tp


In [2]:
valeus = set([True,False])
C = mb.Var('C', valeus)
S = mb.Var('S', valeus) 
R = mb.Var('R', valeus)
W = mb.Var('W', valeus)

tC = {(True,):0.5, (False,):0.5}
PC = mb.Distrib(tC,(C,))

tS_C = {
    (True,): {(True,):0.1, (False,):0.9},
    (False,): {(True,):0.5, (False,):0.5}
}
PS_C = mb.Distrib((tS_C),(S,),(C,))


tR_C = {
    (True,): {(True,):0.8, (False,):0.2},
    (False,): {(True,):0.2, (False,):0.8}
}
PR_C = mb.Distrib((tR_C),(R,),(C,))

tW_SR = {
    (True,True): {(True,):0.99, (False,):0.01},
    (True,False): {(True,):0.9, (False,):0.1},
    (False,True): {(True,):0.9, (False,):0.1},
    (False,False): {(True,):0.0, (False,):0.1}
}
PW_SR = mb.Distrib((tW_SR),(W,),(S,R))

PC.setChildren(set([PS_C,PR_C]))
PS_C.setChildren(set([PW_SR]))
PR_C.setChildren(set([PW_SR]))

vars = set([C,S,R,W])
descomp = tuple([PC,PS_C,PR_C,PW_SR])
PCSRW = mb.Specification(vars,descomp)
Q_CSRW = mb.Question(PCSRW)

In [3]:
Q_CSRW.Query(vars=(R,),indep=(S,W),indep_values=(True,True))

(('R',), (False,), 0.6796116504854368)

In [6]:
Q_CSRW.Query(vars=(R,),indep=(C,S,W),indep_values=(True,False,True), aproximation=True,N=50000)

(('R',), (True,), 0.9957616707616708)

In [5]:
Mb = []
vars_Mb = set([C,S,R])

for d in PCSRW.descomp:
    if d.check(vars_Mb):
        Mb.append(d)

tC_SR = {}
S.event = True
R.event = False

for Cv in C.values:
    C.event = Cv
    p = 1
    for d in Mb:
        p *= d.P()
    tC_SR[(Cv,)] = p

x = 1 / (np.sum(list(tC_SR.values())))
print(tC_SR[(False,)] * x)
print(tC_SR[(True,)] * x)
print((tC_SR[(True,)] * x) + (tC_SR[(False,)] * x))
    

0.9523809523809524
0.04761904761904763
1.0
